In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import datetime
import psycopg2
from model_creation.schemas.topics import username_to_topic
from model_creation.scripts.helpers import create_connection

In [ ]:
def create_topic_data_table():
  #create connection to db
  conn, cursor = create_connection()

  table_check_sql = """
  SELECT EXISTS (
    SELECT FROM pg_tables 
    WHERE schemaname = 'public' 
    AND tablename = 'topic_data'
  );
  """

  create_table_sql = """
    CREATE TABLE topic_data (
      id SERIAL PRIMARY KEY,
      text TEXT NOT NULL,
      source TEXT NOT NULL,
      topic TEXT NOT NULL,
      site TEXT NOT NULL,
      date TIMESTAMP NOT NULL,
      UNIQUE (text)
    );
    """
  try:
    cursor.execute(table_check_sql)
    table_exists = cursor.fetchone()[0]
    if not table_exists:
      cursor.execute(create_table_sql)
      conn.commit()
      print("Table created successfully")
    else:
      print("Table exists")
  except (Exception, psycopg2.DatabaseError) as error:
    print(error)

  cursor.close()
  conn.close()

In [6]:
def get_infinite_scroll_post(url,username,topic, site,target,post_count_break):
  options = Options()
  options.headless = True
  driver = webdriver.ChromiumEdge(options=options)

  driver.get(url)
  insert_sql = """
    INSERT INTO topic_data (
      text, source, topic, site, date
    ) VALUES (
      %s, %s, %s, %s, %s
    )
    """
  conn,cursor = create_connection()
  conn.autocommit = True

  date = datetime.datetime.now()

  seen_posts = set()
  post_count = 0

  while post_count < post_count_break:
    for _ in range(5):
      driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
      time.sleep(0.5)
    elements = driver.find_elements(By.CSS_SELECTOR, target)
    for element in elements:
      content = element.text
      if (len(content.split()) > 4): print(content)
      if (content and content not in seen_posts):
        seen_posts.add(content)
        try:
          cursor.execute(insert_sql,(content,username,topic, site, date))
          post_count+=1
        except(Exception) as error:
          print(error)
          conn.rollback
      if post_count >= post_count_break: break
  
  driver.quit()
  conn.close()
  cursor.close()

In [ ]:
def get_facebook_posts(username, post_count_break):
  url = f"https://www.facebook.com/{username}/posts"
  target = '[data-ad-preview="message"]'
  topic = username_to_topic[username]
  get_infinite_scroll_post(url, username, topic, "facebook", target, post_count_break)

def get_twitter_posts(username, post_count_break):
  url = f"https://www.x.com/{username}"
  target = '[data-testid="tweetText"]'
  topic = username_to_topic[username]
  get_infinite_scroll_post(url, username, topic, "twitter", target, post_count_break)

def get_reddit_posts(username, post_count_break):
  url = f"https://www.reddit.com/t/{username}"
  target = '[slot="full-post-link"]'
  topic = username_to_topic[username]
  get_infinite_scroll_post(url, username, topic, "reddit", target, post_count_break)

def get_subreddit_posts(username, post_count_break):
  url = f"https://www.reddit.com/r/{username}/top/?t=week"
  target = '[slot="full-post-link"]'
  topic = username_to_topic[username]
  get_infinite_scroll_post(url, username, topic, "reddit", target, post_count_break)

In [ ]:
create_topic_data_table()
get_reddit_posts("news_and_politics",post_count_break=100)

First antibiotic in 50 years to tackle superbug in final testing
Meet the Israeli TV Channel that has literally been inciting genocide onto millions of screens in Israel
‘Stateless Overnight’: Kuwait strips over 37,000, mostly women, of citizenship
Rep. Jasmine Crockett Says She'll Run To Be Top Dem In Key Committee: 'I Can Provide What We Need'
North Korea arrests 4th official over failed warship launch
Opinion | Trump admin likely sent legal immigrants to a foreign prison
Dalton College Student Ximena Arias-Cristobal, Released After 2+ Weeks in ICE Custody, Warns: "People Should Fear Being Deported
Video Captures Heartfelt Moments After Ximena Arias-Cristobal's Release After Over Two Weeks in ICE Custody Due to Officer Kevin Hogan Lying About Traffic Stop
Daca negoita și acoliții săi mai stau mult în libertate, pun sectorul pe butuci
Former idf released by Hamas during a ceasefire deal has said one of her biggest fears during captivity were strikes carried out by Israel
Accused neo-N

InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=136.0.7103.114)
Stacktrace:
	GetHandleVerifier [0x00007FF791D9CF45+75717]
	GetHandleVerifier [0x00007FF791D9CFA0+75808]
	(No symbol) [0x00007FF791B68F9A]
	(No symbol) [0x00007FF791B54E35]
	(No symbol) [0x00007FF791B79DB4]
	(No symbol) [0x00007FF791BEEE75]
	(No symbol) [0x00007FF791C0ECC2]
	(No symbol) [0x00007FF791BE7153]
	(No symbol) [0x00007FF791BB0421]
	(No symbol) [0x00007FF791BB11B3]
	GetHandleVerifier [0x00007FF79209D71D+3223453]
	GetHandleVerifier [0x00007FF792097CC2+3200322]
	GetHandleVerifier [0x00007FF7920B5AF3+3322739]
	GetHandleVerifier [0x00007FF791DB6A1A+180890]
	GetHandleVerifier [0x00007FF791DBE11F+211359]
	GetHandleVerifier [0x00007FF791DA5294+109332]
	GetHandleVerifier [0x00007FF791DA5442+109762]
	GetHandleVerifier [0x00007FF791D8BA59+4825]
	BaseThreadInitThunk [0x00007FFD8075E8D7+23]
	Rtl